In [1]:
from numpy import load
import os
import numpy as np
import re
import mir_eval
import csv
import pandas as pd
from statistics import mean

In [2]:
melody_file_path ="/Users/nissimolaji/Desktop/5thyear/Dissertation/Evaluation/ismir2017-deepsalience-master/predict/outputs/melody2/"

In [3]:
ground_truth_melody = "/Users/nissimolaji/Desktop/5thyear/Dissertation/Evaluation/MedleyDB/Melody2/"

In [4]:
file_p ="/Users/nissimolaji/Desktop/5thyear/Dissertation/Evaluation/ismir2017-deepsalience-master/predict/outputs/melody2/MusicDelta_Britpop_MIX_melody2_singlef0.csv"

In [25]:
output_arr=[]
with open(file_p, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        time_stamps = [i.split('\t', 1)[0] for i in row]
        freqs =   [i.split('\t', 1)[1] for i in row]
        
        


# #         [i.split('\t', 1)[0] for i in l]
#         print([i.split('\t', 1)[1] for i in row])

In [27]:
arr=[]
for file in os.listdir(melody_file_path):
    if not file.startswith('.'):
        path = melody_file_path+file
        with open(path, 'r') as f:
            reader = csv.reader(f)
            time=[]
            freq=[]
            for row in reader:
             
                for i in row:                   
                    time_stamps = i.split('\t', 1)[0] 
                    freqs =   i.split('\t', 1)[1] 
                    if int(float(freqs))<0:
                        freqs =0
                    time.append(float(time_stamps))
                    freq.append(float(freqs))
            arr.append(np.array([time,freq,file]))


<ipython-input-27-dd0846902ce4>:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr.append(np.array([time,freq,file]))


In [1]:
# print(arr[0])

In [32]:
# print(arr)

In [24]:
import re
import mir_eval
import csv
import pandas as pd
from statistics import mean

In [29]:
output_arr=[]
for ground_truth in os.listdir(ground_truth_melody):
    for melody_extracted in arr:
        
        if not ground_truth.startswith('.'):
            ground_truth_filename = os.path.splitext(ground_truth)[0]
            melody_extracted_renamed = re.sub(r'_MIX_melody2_singlef0.csv', '', melody_extracted[2], re.I)
            ground_truth_renamed = re.sub(r'_MELODY2', '', ground_truth_filename, re.I)
            
            timestamps_extracted_melody = np.array(melody_extracted[0])
#             print(type(timestamps_extracted_melody))
            freq_extracted_melody=np.array(melody_extracted[1])
    
    
            
            if melody_extracted_renamed==ground_truth_renamed:
                path = ground_truth_melody +ground_truth

                df = pd.read_csv(path)
                df.columns =['Timestamps', 'Freq']
               
                
                timestamps_ground_truth = df['Timestamps'].tolist()
                melody_ground_truth = df['Freq'].tolist()
                timestamps_ground_truth= np.array(timestamps_ground_truth)
                melody_ground_truth=np.array(melody_ground_truth)

#                 print(df)
               
                
                (ref_v, ref_c,est_v, est_c) = mir_eval.melody.to_cent_voicing(timestamps_ground_truth,
                                                     melody_ground_truth,
                                                    timestamps_extracted_melody,
                                                    freq_extracted_melody)
# #   Voicing Recall Rate              
                voicing_recall_rate = mir_eval.melody.voicing_recall(ref_v, est_v)
#     Voicing False Alarm Rate
                voicing_false_alarm_rate=mir_eval.melody.voicing_false_alarm(ref_v, est_v)
#   Raw Pitch Accuracy  
                raw_pitch = mir_eval.melody.raw_pitch_accuracy(ref_v, ref_c,est_v, est_c)
#     Raw Chroma Accuracy
                raw_chroma = mir_eval.melody.raw_chroma_accuracy(ref_v, ref_c,est_v, est_c)
#     Overall Accuracy
                overall_accuracy = mir_eval.melody.overall_accuracy(ref_v, ref_c,est_v, est_c)
                output_arr.append([melody_extracted_renamed,voicing_recall_rate,voicing_false_alarm_rate,raw_pitch,raw_chroma,overall_accuracy])
                
                

                
                
                
                

                

In [30]:
df = pd.DataFrame(output_arr,columns=['File name','Voicing Recall Rate','Voicing False Alarm Rate','Raw Pitch Accuracy','Raw Chroma Accuracy','Overall Accuracy'])

In [31]:
voicing_recall = df['Voicing Recall Rate'].tolist()
voicing_false= df['Voicing False Alarm Rate'].tolist()
raw_pitch_acc = df['Raw Pitch Accuracy'].tolist()
raw_chroma_acc=df['Raw Chroma Accuracy'].tolist()
overall_acc=df['Overall Accuracy'].tolist()


In [32]:
average_voicing_recall = mean(voicing_recall)
average_voicing_false = mean(voicing_false)
average_raw_pitch_accuracy=mean(raw_pitch_acc)
average_raw_chroma_accuracy= mean(raw_chroma_acc)
average_overall_accuracy= mean(overall_acc)

In [33]:
results = [average_voicing_recall,average_voicing_false,average_raw_pitch_accuracy,average_raw_chroma_accuracy,average_overall_accuracy]

In [34]:
df = pd.DataFrame(results,index=['Avg Voicing Recall Rate','Avg Voicing False Alarm Rate','Avg Raw Pitch Accuracy','Avg Raw Chroma Accuracy','Avg Overall Accuracy'])

In [35]:
df

,0
Avg Voicing Recall Rate,0.673081
Avg Voicing False Alarm Rate,0.269724
Avg Raw Pitch Accuracy,0.557460
Avg Raw Chroma Accuracy,0.584868
Avg Overall Accuracy,0.626799
